In [ ]:
%matplotlib inline

Composite failure operator {#failure_operator_example}
==========================

This operator computes the minimum and maximum failure values and
failure modes of a combined failure criterion. A combined failure
criterion is a selection of failure criteria such as Puck, Tsai-Wu, Face
Sheet Wrinkling\...

The combined failure criterion allows you to assess different type of
materials and failure modes at once. The scoping enables to evaluate of
the min and max failure per element, or to select a list of materials or
plies.


Load ansys libraries


In [ ]:
from ansys.dpf.composites.composite_model import CompositeModel, CompositeScope
from ansys.dpf.composites.connect_to_or_start_server import connect_to_or_start_server
from ansys.dpf.composites.enums import FailureOutput
from ansys.dpf.composites.example_helper.example_helper import get_continuous_fiber_example_files
from ansys.dpf.composites.failure_criteria import (
    CombinedFailureCriterion,
    CoreFailureCriterion,
    MaxStrainCriterion,
    MaxStressCriterion,
    VonMisesCriterion,
)

server = connect_to_or_start_server()
composite_files_on_server = get_continuous_fiber_example_files(server, "shell")

Definition of the combined failure criterion


In [ ]:
def get_combined_failure_criterion() -> CombinedFailureCriterion:
    max_strain = MaxStrainCriterion()
    max_stress = MaxStressCriterion()
    core_failure = CoreFailureCriterion()
    von_mises_strain_only = VonMisesCriterion(vme=True, vms=False)

    return CombinedFailureCriterion(
        name="failure of all materials",
        failure_criteria=[max_strain, max_stress, core_failure, von_mises_strain_only],
    )

Setup composite model


In [ ]:
composite_model = CompositeModel(composite_files_on_server, server)
output_all_elements = composite_model.evaluate_failure_criteria(
    combined_criteria=get_combined_failure_criterion(),
)

Plot the max IRF per element


In [ ]:
irf_field = output_all_elements.get_field({"failure_label": FailureOutput.failure_value.value})
irf_field.plot()

Scope failure evaluation to certain element scope


In [ ]:
output_two_elements = composite_model.evaluate_failure_criteria(
    combined_criteria=get_combined_failure_criterion(),
    composite_scope=CompositeScope(elements=[1, 3]),
)
irf_field = output_two_elements.get_field({"failure_label": FailureOutput.failure_value.value})
irf_field.plot()

Scope by plies


In [ ]:
output_woven_plies = composite_model.evaluate_failure_criteria(
    combined_criteria=get_combined_failure_criterion(),
    composite_scope=CompositeScope(plies=["P1L1__ud_patch ns1"]),
)
irf_field = output_woven_plies.get_field({"failure_label": FailureOutput.failure_value.value})
irf_field.plot()